In [1]:
'''
SIGANTEの「【練習問題】自動車の評価」の実装

6つの特徴量からクラスラベル（unacc, acc, good, vgood）を推定する

トレーニングデータ：train_data（train.tsv）

 1. 特徴量の文字列を数値に変換する
 2. トレーニングデータを学習用データ（learning_data）と検証用データ（validation_data）に分割する
 3. 特徴量のスケーリング（標準化）を行う
 4. 学習用データをロジスティック回帰に適用する
'''

In [25]:
# データセットを読み込むためにpandasをインポートする
import pandas as pd

In [26]:
# pandasのread_csvメソッドを用いてトレーニングデータを読み込む
train_data = pd.read_csv('./train.tsv', delimiter='\t')

# pandasのread_csvメソッドを用いてテストデータを読み込む
test_data = pd.read_csv('./test.tsv', delimiter='\t')

In [27]:
train_data

,id,class,buying,maint,doors,persons,lug_boot,safety
0,0,unacc,low,med,3,2,small,low
1,3,acc,low,high,3,more,small,med
2,7,unacc,vhigh,high,5more,2,small,med
3,11,acc,high,high,3,more,big,med
4,12,unacc,high,high,3,2,med,high
...,...,...,...,...,...,...,...,...
859,1721,acc,high,high,5more,more,med,med
860,1722,unacc,high,high,5more,4,small,med
861,1724,unacc,high,low,3,more,med,low
862,1725,unacc,low,low,5more,2,small,med


In [28]:
test_data

,id,buying,maint,doors,persons,lug_boot,safety
0,1,high,vhigh,2,more,big,low
1,2,vhigh,vhigh,5more,4,small,high
2,4,med,high,2,2,small,low
3,5,med,med,4,4,big,med
4,6,high,high,2,4,med,low
...,...,...,...,...,...,...,...
859,1718,low,low,3,2,big,high
860,1719,med,low,5more,2,big,med
861,1720,vhigh,vhigh,3,2,big,med
862,1723,high,vhigh,4,4,small,low


In [29]:
train_data.columns

Index(['id', 'class', 'buying', 'maint', 'doors', 'persons', 'lug_boot',
       'safety'],
      dtype='object')

In [37]:
# トレーニングデータのbuying（車の売値）のユニークな値を取得する
# buyingは順序特徴量（vhigt,high,med,low）である
train_data['buying'].unique()

array(['low', 'vhigh', 'high', 'med'], dtype=object)

In [38]:
# 順序特徴量であるbuyingの変換辞書を設定する
buying_mapping = {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1}

In [39]:
# トレーニングデータのbuyingを数値に変換する
train_data['buying'] = train_data['buying'].map(buying_mapping)

In [46]:
# 順序特徴量であるmaintの変換辞書を設定する
maint_mapping = {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1}

In [47]:
# トレーニングデータのmaintを数値に変換する
train_data['maint'] = train_data['maint'].map(maint_mapping)

In [49]:
# 順序特徴量であるdoorsの変換辞書を設定する
doors_mapping = {'5more': 5, '4': 4, '3': 3, '2': 2}

In [50]:
# トレーニングデータのdoorsを数値に変換する
train_data['doors'] = train_data['doors'].map(doors_mapping)

In [51]:
train_data['persons'].unique()

array(['2', 'more', '4'], dtype=object)

In [52]:
# 順序特徴量であるpersonsの変換辞書を設定する
persons_mapping = {'more': 6, '4': 4, '2': 2}

In [53]:
# トレーニングデータのpersonsを数値に変換する
train_data['persons'] = train_data['persons'].map(persons_mapping)

In [54]:
train_data['lug_boot'].unique()

array(['small', 'big', 'med'], dtype=object)

In [55]:
# 順序特徴量であるlug_bootの変換辞書を設定する
lug_boot_mapping = {'big': 3, 'med': 2, 'small': 1}

In [56]:
# トレーニングデータのlug_bootを数値に変換する
train_data['lug_boot'] = train_data['lug_boot'].map(lug_boot_mapping)

In [57]:
# 順序特徴量であるsafetyの変換辞書を設定する
safety_mapping = {'high': 3, 'med': 2, 'low': 1}

In [58]:
# トレーニングデータのsafetyを数値に変換する
train_data['safety'] = train_data['safety'].map(safety_mapping)

In [61]:
# クラスラベル（class）の変換辞書を設定する
class_mapping = {'vgood': 4, 'good': 3, 'acc': 2, 'unacc': 1}

In [62]:
# トレーニングデータのクラスラベル（class）を数値に変換する
train_data['class'] = train_data['class'].map(class_mapping)

In [64]:
# トレーニングデータが変換されていることを確認する
train_data

,id,class,buying,maint,doors,persons,lug_boot,safety
0,0,1,1,2,3,2,1,1
1,3,2,1,3,3,6,1,2
2,7,1,4,3,5,2,1,2
3,11,2,3,3,3,6,3,2
4,12,1,3,3,3,2,2,3
...,...,...,...,...,...,...,...,...
859,1721,2,3,3,5,6,2,2
860,1722,1,3,3,5,4,1,2
861,1724,1,3,1,3,6,2,1
862,1725,1,1,1,5,2,1,2


In [101]:
# pandasのilocメソッドを用いてトレーニングデータを特徴量（train_data_X）とクラスラベル（train_data_Y）に分離する
# idは学習に用いないため別に保管する
train_data_X = train_data.iloc[:,2:]
train_data_Y = train_data.iloc[:,1:2]
train_data_id = train_data.iloc[:,0:1]

In [107]:
# トレーニングデータを学習用データと検証用データに分割する
from sklearn.model_selection import train_test_split

learning_data_X, validation_data_X, learning_data_Y, validation_data_Y = train_test_split(
    train_data_X, train_data_Y, test_size=0.3, random_state=1, stratify=train_data_Y)

In [108]:
# トレーニングデータの特徴量の標準化を行う
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
stdsc.fit(learning_data_X)

learning_data_X_std = stdsc.transform(learning_data_X)
validation_data_X_std = stdsc.transform(validation_data_X)

In [110]:
# ロジスティック回帰による学習を行う
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(learning_data_X_std, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [111]:
# 検証用データを用いてロジスティック回帰による予測を行う
validation_data_Y_predict = lr.predict(validation_data_X_std)

In [116]:
# 予測値と正解ラベルの比較を行う
import numpy as np

np.concatenate([np.reshape(validation_data_Y_predict, (260, 1)), 
                np.reshape(validation_data_Y, (260, 1))], 1)

array([[1, 1],
       [1, 1],
       [1, 2],
       [2, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 2],
       [1, 1],
       [1, 1],
       [1, 2],
       [2, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [3, 3],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 3],
       [1, 1],
       [2, 2],
       [2, 3],
       [1, 2],
       [1, 1],
       [3, 3],
       [2, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 2],
       [1, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [2, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [2, 2],
       [1, 2],
       [1, 1],
       [2, 2],
       [1, 2],
       [1, 1],
       [1, 1],
       [1, 2],
       [2, 4],
       [2, 2],
       [4, 3],
       [1, 1],
       [1, 2],
       [2, 2],
       [1, 1],
       [2, 2],
       [1,

In [118]:
score = lr.score(validation_data_X_std, validation_data_Y)

In [119]:
score

0.7923076923076923

In [4]:
# データセットを読み込むためにpandasをインポートする
import pandas as pd

# pandasのread_csvメソッドを用いてトレーニングデータ（train.tsv）を読み込む
# トレーニングデータはサイトからダウンロード済み
train_data = pd.read_csv('./train.tsv', delimiter='\t')

# トレーニングデータを表示
train_data

,id,class,buying,maint,doors,persons,lug_boot,safety
0,0,unacc,low,med,3,2,small,low
1,3,acc,low,high,3,more,small,med
2,7,unacc,vhigh,high,5more,2,small,med
3,11,acc,high,high,3,more,big,med
4,12,unacc,high,high,3,2,med,high
...,...,...,...,...,...,...,...,...
859,1721,acc,high,high,5more,more,med,med
860,1722,unacc,high,high,5more,4,small,med
861,1724,unacc,high,low,3,more,med,low
862,1725,unacc,low,low,5more,2,small,med


In [5]:
# 各特徴量の変換用の辞書を設定する
buying_mapping = {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1}
maint_mapping = {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1}
doors_mapping = {'5more': 5, '4': 4, '3': 3, '2': 2}
persons_mapping = {'more': 6, '4': 4, '2': 2}
lug_boot_mapping = {'big': 3, 'med': 2, 'small': 1}
safety_mapping = {'high': 3, 'med': 2, 'low': 1}

# クラスラベル（class）の変換用の辞書を設定する
class_mapping = {'vgood': 4, 'good': 3, 'acc': 2, 'unacc': 1}

# トレーニングデータの各特徴量を変換する
train_data['buying'] = train_data['buying'].map(buying_mapping)
train_data['maint'] = train_data['maint'].map(maint_mapping)
train_data['doors'] = train_data['doors'].map(doors_mapping)
train_data['persons'] = train_data['persons'].map(persons_mapping)
train_data['lug_boot'] = train_data['lug_boot'].map(lug_boot_mapping)
train_data['safety'] = train_data['safety'].map(safety_mapping)

# トレーニングデータのクラスラベル（class）を変換する
train_data['class'] = train_data['class'].map(class_mapping)

# 変換後のトレーニングデータを表示
train_data

,id,class,buying,maint,doors,persons,lug_boot,safety
0,0,1,1,2,3,2,1,1
1,3,2,1,3,3,6,1,2
2,7,1,4,3,5,2,1,2
3,11,2,3,3,3,6,3,2
4,12,1,3,3,3,2,2,3
...,...,...,...,...,...,...,...,...
859,1721,2,3,3,5,6,2,2
860,1722,1,3,3,5,4,1,2
861,1724,1,3,1,3,6,2,1
862,1725,1,1,1,5,2,1,2


In [6]:
# pandasのilocメソッドを用いてトレーニングデータを特徴量（train_data_X）とクラスラベル（train_data_Y）に分離する
# idは学習に用いないため別に保管する
train_data_X = train_data.iloc[:,2:]
train_data_Y = train_data.iloc[:,1:2]
train_data_id = train_data.iloc[:,0:1]

In [7]:
from sklearn.model_selection import train_test_split

# トレーニングデータを学習用データ（learning_data）と検証用データ（learning_data）に分割する
learning_data_X, validation_data_X, learning_data_Y, validation_data_Y = train_test_split(
    train_data_X, train_data_Y, test_size=0.3, random_state=1, stratify=train_data_Y)

In [8]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()

stdsc.fit(learning_data_X)

# トレーニングデータの特徴量の標準化を行う
learning_data_X_std = stdsc.transform(learning_data_X)
validation_data_X_std = stdsc.transform(validation_data_X)

In [9]:
learning_data_X

,buying,maint,doors,persons,lug_boot,safety
143,3,2,5,4,1,3
528,1,1,3,6,2,3
254,3,1,5,4,3,3
625,2,4,4,6,2,3
307,4,2,3,4,3,2
...,...,...,...,...,...,...
834,4,3,3,2,1,1
612,3,1,4,4,2,2
418,3,3,5,2,3,1
343,4,4,5,2,1,3


In [10]:
learning_data_X_std

array([[ 0.50664927, -0.48434529,  1.36253261, -0.01985832, -1.23751095,
         1.17171256],
       [-1.30409723, -1.38172122, -0.47856291,  1.1795842 , -0.01016019,
         1.17171256],
       [ 0.50664927, -1.38172122,  1.36253261, -0.01985832,  1.21719058,
         1.17171256],
       ...,
       [ 0.50664927,  0.41303064,  1.36253261, -1.21930084,  1.21719058,
        -1.27712615],
       [ 1.41202251,  1.31040657,  1.36253261, -1.21930084, -1.23751095,
         1.17171256],
       [-0.39872398, -1.38172122, -0.47856291, -1.21930084, -1.23751095,
        -0.05270679]])

In [1]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)

# 標準化した学習用データを用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X_std, learning_data_Y)

NameError: name 'learning_data_X_std' is not defined

In [14]:
score = lr.score(validation_data_X_std, validation_data_Y)
score

0.7769230769230769